# Chapter 7: Ensemble Learning dan Random Forests

Bab ini membahas pendekatan *Ensemble Learning*, yaitu teknik Machine Learning yang menggabungkan beberapa model untuk meningkatkan akurasi dan stabilitas prediksi. Pendekatan ini banyak digunakan dalam sistem produksi dan kompetisi Machine Learning karena mampu mengurangi kesalahan generalisasi.

Alih-alih mengandalkan satu model tunggal, Ensemble Learning memanfaatkan kekuatan kolektif dari beberapa model yang memiliki karakteristik berbeda.


## 1. Konsep Dasar Ensemble Learning

Ensemble Learning didasarkan pada prinsip bahwa sekumpulan model yang beragam sering kali mampu menghasilkan prediksi yang lebih baik dibandingkan satu model tunggal.

Dalam Ensemble Learning:
- Model individual disebut *base learner*
- Gabungan beberapa model disebut *ensemble*
- Prediksi akhir diperoleh melalui agregasi hasil prediksi


### 1.1 Weak Learner dan Strong Learner

Weak learner adalah model dengan performa sedikit lebih baik dari tebakan acak. Meskipun lemah secara individual, kombinasi banyak weak learner dapat menghasilkan strong learner yang sangat akurat.

Keberhasilan pendekatan ini sangat bergantung pada keberagaman kesalahan antar model.


## 2. Voting Classifier

Voting Classifier merupakan bentuk Ensemble Learning paling sederhana. Beberapa model dilatih secara terpisah, kemudian prediksinya digabungkan menggunakan mekanisme pemungutan suara.


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

log_reg = LogisticRegression()
rf_clf = RandomForestClassifier()
svm_clf = SVC()

voting_hard = VotingClassifier(
    estimators=[
        ('lr', log_reg),
        ('rf', rf_clf),
        ('svm', svm_clf)
    ],
    voting='hard'
)


### 2.1 Evaluasi Voting Classifier

Untuk mengevaluasi performa Voting Classifier, digunakan dataset sintetis dan metrik akurasi.


In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=10000, noise=0.35, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

for model in (log_reg, rf_clf, svm_clf, voting_hard):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.__class__.__name__, accuracy_score(y_test, y_pred))


### 2.2 Soft Voting

Soft voting menggunakan probabilitas prediksi dari setiap model. Probabilitas tersebut dirata-ratakan, dan kelas dengan probabilitas tertinggi dipilih sebagai hasil akhir.


In [ ]:
svm_clf = SVC(probability=True)

voting_soft = VotingClassifier(
    estimators=[
        ('lr', log_reg),
        ('rf', rf_clf),
        ('svm', svm_clf)
    ],
    voting='soft'
)


## 3. Bagging dan Pasting

Bagging dan Pasting merupakan teknik Ensemble Learning yang meningkatkan diversitas model dengan melatih setiap model pada subset data yang berbeda.


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=300,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1
)


In [ ]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)


## 4. Out-of-Bag (OOB) Evaluation

Out-of-Bag evaluation memungkinkan estimasi performa model tanpa validation set terpisah, dengan memanfaatkan data yang tidak terpilih selama proses bootstrap sampling.


In [ ]:
bag_oob = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=300,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1
)

bag_oob.fit(X_train, y_train)
bag_oob.oob_score_


## 5. Random Forest

Random Forest merupakan ensemble Decision Tree yang dilatih menggunakan Bagging dengan tambahan randomisasi fitur pada setiap split.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)

rf_model.fit(X_train, y_train)


## 6. Boosting

Boosting melatih model secara berurutan, di mana setiap model baru berfokus memperbaiki kesalahan model sebelumnya.


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    learning_rate=0.5,
    random_state=42
)

ada_clf.fit(X_train, y_train)


## Closing Summary (Chapter 7)

Bab ini membahas berbagai teknik Ensemble Learning, termasuk Voting, Bagging, Random Forest, Boosting, dan Stacking. Pendekatan ini menunjukkan bahwa kombinasi beberapa model yang beragam mampu menghasilkan sistem Machine Learning yang lebih kuat, stabil, dan akurat dibandingkan model tunggal.
